1. Install datasets and confirm import


GoEmotions Dataset
------------------

Description:
    A corpus of 58,000 curated Reddit comments annotated with 27 fine-grained emotion categories and a neutral label. 
    Used to train emotion classification models.

Labels:
    - 27 emotions: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval,
      disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse,
      sadness, surprise
    - neutral

License:
    Creative Commons Attribution 4.0 International

Citation:
    Demszky et al. (2020). GoEmotions: A Dataset of Fine-Grained Emotions. https://arxiv.org/abs/2005.00547

Source:
    https://github.com/google-research/google-research/tree/master/goemotions

Provider:
    Google Research

Recommended Usage:
    Suitable for fine-tuning transformer models for multi-class or multi-label emotion classification.



In [1]:
from datasets import load_dataset


/home/selfhealer-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Load the raw GoEmotions dataset

In [2]:
import pandas as pd

# Load the main file
df = pd.read_csv("data/full_dataset/goemotions_1.csv", names=["text", "labels", "id"])

# Take a look
df.head()


/tmp/ipykernel_20284/1507003249.py:4: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/full_dataset/goemotions_1.csv", names=["text", "labels", "id"])


text  \
text                                               id      author        subreddit        link_id   parent_id  created_utc  rater_id example_very_unclear admiration amusement anger annoyance approval caring confusion curiosity desire disappointment disapproval disgust embarrassment excitement fear gratitude grief joy love nervousness optimism pride realization relief remorse  sadness   
That game hurt.                                    eew5j0j Brdd9         nrl              t3_ajis4z t1_eew18eq 1548381039.0 1        False                0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0              1   
 >sexuality shouldn’t be a grouping category It... eemcysk TheGreen888   unpopularopinion t3_ai4q37 t3_ai4q37  1548084169.0 37       True                 0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0              0   
You do right, if you don't care then fuck 'em!     ed2mah1 Labalool      confessions      t3_abru74 t1_ed2m7g7 1546427744.0 37       False                0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0              0   
Man I love reddit.                                 eeibobj MrsRobertshaw facepalm         t3_ahulml t3_ahulml  1547965054.0 18       False                0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   1    0           0        0     0           0      0              0   

                                                                                                                                                                                                                                                                                                                                                                                             labels  \
text                                               id      author        subreddit        link_id   parent_id  created_utc  rater_id example_very_unclear admiration amusement anger annoyance approval caring confusion curiosity desire disappointment disapproval disgust embarrassment excitement fear gratitude grief joy love nervousness optimism pride realization relief remorse  surprise   
That game hurt.                                    eew5j0j Brdd9         nrl              t3_ajis4z t1_eew18eq 1548381039.0 1        False                0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0               0   
 >sexuality shouldn’t be a grouping category It... eemcysk TheGreen888   unpopularopinion t3_ai4q37 t3_ai4q37  1548084169.0 37       True                 0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0               0   
You do right, if you don't care then fuck 'em!     ed2mah1 Labalool      confessions      t3_abru74 t1_ed2m7g7 1546427744.0 37       False                0          0         0     0         0        0      0         0         0      0              0           0       0             0          0    0         0     0   0    0           0        0     0           0      0               0   
Man I love reddit.                                 eeibobj MrsRobertshaw facepalm     

**Columns Breakdown:**
text → the Reddit comment

id, author, subreddit, etc. → metadata (can be mostly ignore)

The emotion columns (admiration, amusement, etc.) → binary values (0 or 1) indicating if the label applies

We’ll now directly convert 1s in emotion columns into emotion names, then group them into positive / negative / neutral buckets.

## Extract Emotion Labels per Row

In [3]:
# Full emotion + metadata column headers as per Google GoEmotions schema. Defining the emotions category
columns = [
    "text", "id", "author", "subreddit", "link_id", "parent_id", "created_utc",
    "rater_id", "example_very_unclear", "admiration", "amusement", "anger",
    "annoyance", "approval", "caring", "confusion", "curiosity", "desire",
    "disappointment", "disapproval", "disgust", "embarrassment", "excitement",
    "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism",
    "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

# Loading the CSV with proper headers
df = pd.read_csv("data/full_dataset/goemotions_1.csv", names=columns, skiprows=1)

# All 28 emotion columns. Getting the emotions active in each row
emotion_columns = columns[9:]  

def extract_labels(row):
    return [emotion for emotion in emotion_columns if row[emotion] == 1]

df["emotions"] = df.apply(extract_labels, axis=1)
df[["text", "emotions"]].head()


text   emotions
0                                    That game hurt.  [sadness]
1   >sexuality shouldn’t be a grouping category I...         []
2     You do right, if you don't care then fuck 'em!  [neutral]
3                                 Man I love reddit.     [love]
4  [NAME] was nowhere near them, he was by the Fa...  [neutral]

## Remapping Emotions to positive, negative, neutral

We’ll define three buckets and map each row’s emotions accordingly:

In [4]:
# Define emotion categories per class
positive = {
    "admiration", "amusement", "approval", "gratitude", "joy", "love", 
    "optimism", "pride", "relief", "excitement", "caring"
}
negative = {
    "anger", "annoyance", "disappointment", "disapproval", "disgust", 
    "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"
}
neutral = {
    "neutral", "confusion", "curiosity", "desire", "realization", "surprise"
}

# Remap function
def map_to_class(emotions):
    if any(e in positive for e in emotions):
        return "positive"
    elif any(e in negative for e in emotions):
        return "negative"
    else:
        return "neutral"

# Apply to the dataframe
df["label_text"] = df["emotions"].apply(map_to_class)
df[["text", "emotions", "label_text"]].head()


text   emotions label_text
0                                    That game hurt.  [sadness]   negative
1   >sexuality shouldn’t be a grouping category I...         []    neutral
2     You do right, if you don't care then fuck 'em!  [neutral]    neutral
3                                 Man I love reddit.     [love]   positive
4  [NAME] was nowhere near them, he was by the Fa...  [neutral]    neutral

## Balancing the Dataset

We need to balance the dataset as “neutral” often dominates, which could bias the model. We’ll sample an equal number of examples from each class for fair training.

In [5]:
from sklearn.utils import resample

# Split by class
df_pos = df[df["label_text"] == "positive"]
df_neg = df[df["label_text"] == "negative"]
df_neu = df[df["label_text"] == "neutral"]

# Downsample to the smallest class
min_size = min(len(df_pos), len(df_neg), len(df_neu))
df_balanced = pd.concat([
    resample(df_pos, n_samples=min_size, random_state=42),
    resample(df_neg, n_samples=min_size, random_state=42),
    resample(df_neu, n_samples=min_size, random_state=42)
])

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_balanced["label_text"].value_counts()


label_text
neutral     16080
positive    16080
negative    16080
Name: count, dtype: int64

We now have:

- Clean text column

- Extracted fine-grained emotions

- Remapped label_text into 3 classes

- Balanced samples across all categories

Let's move into Hugging Face Dataset conversion + tokenization, which sets the stage for training the DistilBERT classifier.

## Prepare Dataset for Fine-Tuning

This provides a preprocessed, tokenized dataset that is fully prepared for immediate training, incorporating all essential components.

In [6]:
#! /home/selfhealer-env/bin/python -m pip install transformers==4.46.3 --no-cache-dir --force-reinstall


In [7]:
import transformers
print(transformers.__version__)


4.46.3


In [8]:
import sys
print(sys.executable)


/home/selfhealer-env/bin/python


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "models/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at models/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


So we have the set up to run fine-tuning via Hugging Face’s Trainer

In [10]:
from datasets import Dataset
from transformers import AutoTokenizer

# Encode class labels
label_map = {"positive": 0, "negative": 1, "neutral": 2}
df_balanced["label"] = df_balanced["label_text"].map(label_map)

# Create Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df_balanced[["text", "label"]])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("models/distilbert-base-uncased")

# Tokenize function
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenizer
hf_dataset = hf_dataset.map(tokenize, batched=True)

# Train/test split
hf_dataset = hf_dataset.train_test_split(test_size=0.15)
hf_dataset


Map: 100%|██████████| 48240/48240 [00:04<00:00, 11207.49 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 41004
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7236
    })
})

## Training Setup

We will be using Trainer from Hugging Face Transformers, as it is:

- Offline Friendly: We have the tokenizer, model, and dataset locally — and Trainer can run fully offline.

- Minimal Setup: No need to manually code backprop, optimizer steps, or logging.

- Metrics + Logging Built-in: Accuracy, F1, loss — easy to plug in, and you can direct logs to a .log file for DAG.

- Model Saving & Resuming: Handles checkpoint saving automatically, which is critical for recoverability in a self-healing pipeline.

For the fine-tuning loop, here’s a complete Trainer setup that works with offline DistilBERT model, tokenized dataset, and 3-label classification task. It includes:

- Training config

- Metric computation (accuracy + macro F1)

- Logging

- Model checkpoint saving


#### Metric Setup

In [11]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_macro": f1_score(labels, predictions, average="macro"),
    }


#### Training Arguments

In [12]:
import transformers
print(transformers.__version__)


4.46.3


In [13]:

# !pip install --upgrade transformers

In [14]:
# import transformers
# print(transformers.__version__)


In [15]:
# from transformers import TrainingArguments
# print(TrainingArguments.__module__)


In [16]:
# import transformers
# print(transformers.__file__)


In [17]:
#!pip install --upgrade transformers --no-cache-dir


In [18]:
# import transformers
# print(transformers.__version__)


In [19]:
# from transformers import TrainingArguments
# print(TrainingArguments.__module__)
# print(TrainingArguments.__init__.__code__.co_varnames)


In [20]:
# !pip uninstall -y transformers


In [21]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [22]:
# !rm -rf /home/selfhealer-env/lib/python3.10/site-packages/transformers*


In [23]:
# !pip install transformers==4.52.4 --no-cache-dir --force-reinstall


In [24]:
# !pip install transformers==4.46.3 --no-cache-dir --force-reinstall


In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    eval_strategy="epoch",         
    save_strategy="epoch",        
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_total_limit=2,
)


In [28]:
# from transformers import TrainingArguments
# help(TrainingArguments)


#### Initializing the Trainer to kick off training

Connecting the model, data, tokenizer, and training config together

In [29]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }


In [31]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics, 
)


#### Start Training

In [32]:
trainer.train()


TrainOutput(global_step=7689, training_loss=0.5702699061440993, metrics={'train_runtime': 737.9552, 'train_samples_per_second': 166.693, 'train_steps_per_second': 10.419, 'total_flos': 4073842560771072.0, 'train_loss': 0.5702699061440993, 'epoch': 3.0})

In [33]:
#### Evaluating the Model

In [34]:
trainer.evaluate()


{'eval_loss': 0.7160782217979431,
 'eval_accuracy': 0.7519347705914871,
 'eval_f1_macro': 0.7499544761681735,
 'eval_runtime': 11.8483,
 'eval_samples_per_second': 610.721,
 'eval_steps_per_second': 19.159,
 'epoch': 3.0}

In [35]:
model.save_pretrained("./model_output")
tokenizer.save_pretrained("./model_output")


('./model_output/tokenizer_config.json',
 './model_output/special_tokens_map.json',
 './model_output/vocab.txt',
 './model_output/added_tokens.json',
 './model_output/tokenizer.json')